# Import packages

In [1]:
# import packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# packages for building the model
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras import losses 
from tensorflow.keras import metrics
# packages for training model
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
# misc package
import category_encoders as category_encoder

# Import and view raw data

In [2]:
data = pd.read_csv('heart.csv')

In [3]:
data.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [5]:
# 918 samples, 12 features
data.shape

(918, 12)

# Preprocessing of Data
- Check for null values
- Check for duplicated values
- convert categorical data to numerical data
- Normalize data
- Split data into train and test samples/labels
- Convert datasets to numpy arrays to be able to pass data to Keras models

## Preprocessing data: Check for null and/or duplicated values

In [6]:
# check for missing values - there is no missing data
data.isnull().sum(axis=0)

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [7]:
# check for duplicate values - there is no duplicated data
data.duplicated().sum(axis=0)

0

## Preprocessing data: View unique values for categorical features

In [8]:
category_columns = []

# viewing the unique values, number of dimensions, and shape of each column in the data frame
for col in data.columns:
    if data[col].dtype == 'object':
        print(f'{col}')
        print(f'Values: {data[col].unique()}')
        print('\n')
        category_columns.append(col)
print(f'The following categories: {category_columns} shall be converted to numerical data via pd.get_dummies')

Sex
Values: ['M' 'F']


ChestPainType
Values: ['ATA' 'NAP' 'ASY' 'TA']


RestingECG
Values: ['Normal' 'ST' 'LVH']


ExerciseAngina
Values: ['N' 'Y']


ST_Slope
Values: ['Up' 'Flat' 'Down']


The following categories: ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'] shall be converted to numerical data via pd.get_dummies


In [9]:
# labels - 0 does not have heart disease, 1 does have heart disease
data['HeartDisease'].unique()

array([0, 1])

## Preprocessing data: Converting categorical values into numerical values

In [10]:
# Convert the nomimnal categorical label values to numerical values

# get_dummies method
mod_data = data.copy()
mod_data = pd.get_dummies(mod_data, columns=category_columns)
mod_data.info(), mod_data.head(2)

# encoder = category_encoder.BinaryEncoder(cols = category_columns)
# data_mod = data.copy()
# df_category_encorder = encoder.fit_transform(data_mod)
# df_category_encorder

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    int64  
 1   RestingBP          918 non-null    int64  
 2   Cholesterol        918 non-null    int64  
 3   FastingBS          918 non-null    int64  
 4   MaxHR              918 non-null    int64  
 5   Oldpeak            918 non-null    float64
 6   HeartDisease       918 non-null    int64  
 7   Sex_F              918 non-null    uint8  
 8   Sex_M              918 non-null    uint8  
 9   ChestPainType_ASY  918 non-null    uint8  
 10  ChestPainType_ATA  918 non-null    uint8  
 11  ChestPainType_NAP  918 non-null    uint8  
 12  ChestPainType_TA   918 non-null    uint8  
 13  RestingECG_LVH     918 non-null    uint8  
 14  RestingECG_Normal  918 non-null    uint8  
 15  RestingECG_ST      918 non-null    uint8  
 16  ExerciseAngina_N   918 non

(None,
    Age  RestingBP  Cholesterol  FastingBS  MaxHR  Oldpeak  HeartDisease  \
 0   40        140          289          0    172      0.0             0   
 1   49        160          180          0    156      1.0             1   
 
    Sex_F  Sex_M  ChestPainType_ASY  ...  ChestPainType_NAP  ChestPainType_TA  \
 0      0      1                  0  ...                  0                 0   
 1      1      0                  0  ...                  1                 0   
 
    RestingECG_LVH  RestingECG_Normal  RestingECG_ST  ExerciseAngina_N  \
 0               0                  1              0                 1   
 1               0                  1              0                 1   
 
    ExerciseAngina_Y  ST_Slope_Down  ST_Slope_Flat  ST_Slope_Up  
 0                 0              0              0            1  
 1                 0              0              1            0  
 
 [2 rows x 21 columns])

In [11]:
mod_data.columns

Index(['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak',
       'HeartDisease', 'Sex_F', 'Sex_M', 'ChestPainType_ASY',
       'ChestPainType_ATA', 'ChestPainType_NAP', 'ChestPainType_TA',
       'RestingECG_LVH', 'RestingECG_Normal', 'RestingECG_ST',
       'ExerciseAngina_N', 'ExerciseAngina_Y', 'ST_Slope_Down',
       'ST_Slope_Flat', 'ST_Slope_Up'],
      dtype='object')

In [12]:
features = [col for col in mod_data.columns if col != 'HeartDisease']
output = ['HeartDisease']
features, len(features)

(['Age',
  'RestingBP',
  'Cholesterol',
  'FastingBS',
  'MaxHR',
  'Oldpeak',
  'Sex_F',
  'Sex_M',
  'ChestPainType_ASY',
  'ChestPainType_ATA',
  'ChestPainType_NAP',
  'ChestPainType_TA',
  'RestingECG_LVH',
  'RestingECG_Normal',
  'RestingECG_ST',
  'ExerciseAngina_N',
  'ExerciseAngina_Y',
  'ST_Slope_Down',
  'ST_Slope_Flat',
  'ST_Slope_Up'],
 20)

## Preprocessing data: Convert data to numpy arrays

In [13]:
X = np.array(mod_data[features])
y = np.array(mod_data[output])

len(features), X.shape, y.shape

(20, (918, 20), (918, 1))

In [14]:
type(X), type(y), X.dtype, y.dtype

(numpy.ndarray, numpy.ndarray, dtype('float64'), dtype('int64'))

## Preprocessing data: Splitting data into train and test samples/labels

In [15]:
# split dataframe into train samples/labels and test samples/labels
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, test_size=0.20)
# convert the dataframes to numpy arrays (tensors)
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)

train_samples.shape, train_labels.shape, len(features)

((734, 20), (734, 1), 20)

# Build the Neural Network Model with Keras Sequential class

In [16]:
# building a Sequential model - linear stack of layers
# init the model
model = Sequential()
# add Dense layers to the models
# units = number of nodes, input_shape = tensor shape the input layer expect (inits weights); activation - activation function
model.add(Dense(units=16, activation='relu', input_shape=(20, )))
model.add(Dense(units=16, activation='relu'))
# use sigmoid in last layer because it is a binary classification problem
model.add(Dense(units=1, activation='sigmoid'))

## Visualization of Neural Network Dense Layers

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 625
Trainable params: 625
Non-trainable params: 0
_________________________________________________________________


# Compile the Model
- Assign loss function: the function that is minimized by the optimizer
- Assign optimizer function: how the model learns and minimizes the loss function 
- Choose metrics: used to evaluate the performance of the model

In [18]:
# compilation step - 1) loss function 2) optimizer 3) Metrics to monitor during training and testing
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001),loss=losses.binary_crossentropy,metrics=["acc"])

# alt ways to construct optimizer
# opt = Adam(learning_rate=0.01) # defining the optimizer
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
- Assign the train samples as x with their train labels as y
- Assign batch size, the number of samples that will be propagated through the network
- Assign epochs, the number of iterations the model will run through the layers

In [19]:
# training the model by fitting the normalized training data
history = model.fit(x=train_samples, y=train_labels, batch_size=20, epochs=30)
history

Epoch 1/30
37/37 [==============================] - 1s 839us/step - loss: 2.2997 - acc: 0.6935
Epoch 2/30
37/37 [==============================] - 0s 993us/step - loss: 1.9204 - acc: 0.6853
Epoch 3/30
37/37 [==============================] - 0s 1ms/step - loss: 1.6935 - acc: 0.6880
Epoch 4/30
37/37 [==============================] - 0s 993us/step - loss: 1.3809 - acc: 0.7166
Epoch 5/30
37/37 [==============================] - 0s 820us/step - loss: 1.0859 - acc: 0.6948
Epoch 6/30
37/37 [==============================] - 0s 1ms/step - loss: 1.0055 - acc: 0.6935
Epoch 7/30
37/37 [==============================] - 0s 964us/step - loss: 0.8901 - acc: 0.7030
Epoch 8/30
37/37 [==============================] - 0s 925us/step - loss: 0.8764 - acc: 0.6935
Epoch 9/30
37/37 [==============================] - 0s 852us/step - loss: 0.7120 - acc: 0.7166
Epoch 10/30
37/37 [==============================] - 0s 892us/step - loss: 0.6887 - acc: 0.7003
Epoch 11/30
37/37 [==============================] - 

# Evaluate the Model's performance
- The evaluate method takes in a test sample numpy array and their associated test labels
- Returns the loss value & metrics value (accuracy score) for the model in test mode
- Loss is the scalar value that is attempted to be minimized during training of the model. The lower the loss, the closer our predictions are to the true labels.

In [20]:
results = model.evaluate(test_samples, test_labels)
results

6/6 [==============================] - 0s 879us/step - loss: 0.7382 - acc: 0.7174


[0.7382122278213501, 0.717391312122345]

# Further Experiments
- ## Hidden Layers
    - Try using one or three hidden layers, and see how doing so affects validation and test accuracy.
- ## Hidden Units
    - Try using layers with more hidden units or fewer hidden units: 32 units, 64 units, and so on. 
- ## Loss Functions
    - Try using the mse loss function instead of binary_crossentropy. 
- ## Activation Function
    - Try using the tanh activation (an activation that was popular in the early days of neural networks)

## Reusable function for creating Keras Sequential Models

In [22]:
DEFAULT_LAYERS = [
          Dense(units=16, activation='relu', input_shape=(20, )),
          Dense(units=32, activation='relu'),
          Dense(units=1, activation='sigmoid')
]
DEFAULT_OPTIMIZER = optimizers.RMSprop(learning_rate=0.001)
DEFAULT_LOSS = losses.binary_crossentropy
DEFAULT_METRICS = ["acc"]

In [23]:
def create_neural_network(train_samples, 
                          train_labels,
                          layers = DEFAULT_LAYERS, 
                          optimizer=DEFAULT_OPTIMIZER,
                          loss=DEFAULT_LOSS,
                          metrics=DEFAULT_METRICS):
    model = Sequential(layers)
    model.compile(optimizer=optimizer, loss=loss,metrics=metrics)
    model.fit(x=train_samples, y=train_labels, batch_size=20, epochs=30, verbose=0)
    return model

def get_nn_results(model, test_samples, test_labels):
    return model.evaluate(test_samples, test_labels)

## Further Experiments: Hidden Layers

In [24]:
layers_dict = {
    # 1 hidden layer
    '1': [
          Dense(units=1, activation='sigmoid')
         ],
    # 2 hidden layers
    '2': [
          Dense(units=16, activation='relu', input_shape=(20, )), 
          Dense(units=1, activation='sigmoid')
         ],
    # 3 hidden layers
    '3': [
          Dense(units=16, activation='relu', input_shape=(20, )), 
          Dense(units=16, activation='relu'),
          Dense(units=1, activation='sigmoid')
         ]
}
results_layers_dict = {}

In [25]:
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, test_size=0.20)
# convert the dataframes to numpy arrays (tensors)
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)

for key, layers in layers_dict.items():
    curr_model = create_neural_network(train_samples, train_labels, layers)
    results_layers_dict[key] = get_nn_results(curr_model, test_samples, test_labels)

print('\n')
for key, value in results_layers_dict.items():
    print(f'Achieved {value[0]:.4f} loss {value[1]:.4f} accuracy with {key} layer(s)')

6/6 [==============================] - 0s 1ms/step - loss: 0.4391 - acc: 0.8370


Achieved 1.6108 loss 0.7011 accuracy with 1 layer(s)
Achieved 0.3959 loss 0.8152 accuracy with 2 layer(s)
Achieved 0.4391 loss 0.8370 accuracy with 3 layer(s)


## Further Experiments: Hidden Units

In [26]:
units_dict = {
    # 16 hidden layer
    '16': [
          Dense(units=16, activation='relu', input_shape=(20, )), 
          Dense(units=16, activation='relu'),
          Dense(units=1, activation='sigmoid')
         ],
    # 32 hidden layers
    '32': [
          Dense(units=32, activation='relu', input_shape=(20, )), 
          Dense(units=32, activation='relu'),
          Dense(units=1, activation='sigmoid')
         ],
    # 64 hidden layers
    '64': [
          Dense(units=64, activation='relu', input_shape=(20, )), 
          Dense(units=64, activation='relu'),
          Dense(units=1, activation='sigmoid')
         ]
}
results_units_dict = {}

In [27]:
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, test_size=0.20)
# convert the dataframes to numpy arrays (tensors)
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)

for key, layers in units_dict.items():
    curr_model = create_neural_network(train_samples, train_labels, layers)
    results_units_dict[key] = get_nn_results(curr_model, test_samples, test_labels)

print('\n')
for key, value in results_units_dict.items():
    print(f'Achieved {value[0]:.4f} loss {value[1]:.4f} accuracy with {key} units/nodes per deep layer')

6/6 [==============================] - 0s 689us/step - loss: 0.9110 - acc: 0.6196


Achieved 0.4622 loss 0.7935 accuracy with 16 units/nodes per deep layer
Achieved 0.4520 loss 0.8315 accuracy with 32 units/nodes per deep layer
Achieved 0.9110 loss 0.6196 accuracy with 64 units/nodes per deep layer


## Further Experiments: Loss Functions

In [28]:
loss_functions = ['binary_crossentropy', 'hinge', 'squared_hinge']
results_loss_dict = {}

In [29]:
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, test_size=0.20)
# convert the dataframes to numpy arrays (tensors)
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)

for loss_func in loss_functions:
    curr_model = create_neural_network(train_samples, train_labels, loss=loss_func)
    results_loss_dict[loss_func] = get_nn_results(curr_model, test_samples, test_labels)

print('\n')
for key, value in results_loss_dict.items():
    print(f'Achieved {value[0]:.4f} loss {value[1]:.4f} accuracy with the {key} loss function')

6/6 [==============================] - 0s 981us/step - loss: 0.7773 - acc: 0.7554


Achieved 0.5507 loss 0.7826 accuracy with the binary_crossentropy loss function
Achieved 0.7288 loss 0.7609 accuracy with the hinge loss function
Achieved 0.7773 loss 0.7554 accuracy with the squared_hinge loss function


## Further Experiments: Activation Functions

In [30]:
activation_functions_dict = {
    # sigmoid
    'sigmoid': [
          Dense(units=16, activation='relu', input_shape=(20, )), 
          Dense(units=16, activation='relu'),
          Dense(units=1, activation='sigmoid')
         ],
    # tanh
    'tanh': [
          Dense(units=16, activation='relu', input_shape=(20, )), 
          Dense(units=16, activation='relu'),
          Dense(units=1, activation='tanh')
         ],
    # relu
    'relu': [
          Dense(units=16, activation='relu', input_shape=(20, )), 
          Dense(units=16, activation='relu'),
          Dense(units=1, activation='relu')
         ]
}
results_activation_functions_dict = {}

In [31]:
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, test_size=0.20)
# convert the dataframes to numpy arrays (tensors)
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)

for key, layers in activation_functions_dict.items():
    curr_model = create_neural_network(train_samples, train_labels, layers)
    results_activation_functions_dict[key] = get_nn_results(curr_model, test_samples, test_labels)

print('\n')
for key, value in results_activation_functions_dict.items():
    print(f'Achieved {value[0]:.4f} loss {value[1]:.4f} accuracy with the {key} activation function.')

6/6 [==============================] - 0s 766us/step - loss: 7.7904 - acc: 0.4891


Achieved 0.3497 loss 0.8804 accuracy with the sigmoid activation function.
Achieved 7.7904 loss 0.4891 accuracy with the tanh activation function.
Achieved 7.7904 loss 0.4891 accuracy with the relu activation function.
